In [13]:
import pandas as pd
base_path="/home/jacob/data/movies/dailyboxoffice"
ds_nodash="20240101"
svbase_path =  "/home/jacob/data/movies/merge/dailyboxoffice"
save_path = f"{svbase_path}/dt={ds_nodash}/merged.parquet"
drop_col = ['rnum', 'rank', 'rankInten', 'salesShare']
full_col = ['rnum', 'rank', 'rankInten', 'salesAmt', 'audiCnt',
            'audiAcc', 'scrnCnt', 'showCnt', 'salesShare', 'salesInten',
            'salesChange', 'audiInten', 'audiChange']
    
df=pd.read_parquet("/home/jacob/data/movies/dailyboxoffice/dt=20240101")
df.drop(columns=['rank', 'rnum', 'rankInten', 'salesShare'])
fil_movieCd=[]

for _, row in df.iterrows():
    if pd.isna(row['multiMovieYn']) or pd.isna(row['repNationCd']):
        fil_movieCd.append(row['movieCd'])
        
def merge_values(series):
    return ', '.join(series.dropna().astype(str).unique())

merged_list=[]

for i in set(fil_movieCd):
    fil_dup=df[df['movieCd'] == i][['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd','audiCnt','rnum']]
    if len(fil_dup) == 1 and fil_dup[['multiMovieYn', 'repNationCd']].isna().all(axis=1).iloc[0]:
        fil_dup = fil_dup.fillna("Unclassified")
        merged_list.append(fil_dup)
    else:
        fil_dup=fil_dup.dropna(subset=['multiMovieYn', 'repNationCd'], how='all')
        merged_df = fil_dup.groupby(['movieCd', 'movieNm'], as_index=False).agg({
            'multiMovieYn': merge_values,
            'repNationCd': merge_values,
            'audiCnt': 'max'
        })
        merged_list.append(merged_df)
f_merged_df = pd.concat(merged_list, ignore_index=True)
f_merged_df['rank'] =f_merged_df['audiCnt'].rank(ascending=False,method='dense')
unique_df_sorted = f_merged_df.sort_values(by='rank')
unique_df_sorted[['multiMovieYn', 'repNationCd']] = unique_df_sorted[['multiMovieYn', 'repNationCd']].replace('', pd.NA)
unique_df_sorted


,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
22,20203702,노량: 죽음의 바다,N,K,290717.0,1.0
2,20212866,서울의 봄,N,K,262422.0,2.0
21,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,58239.0,3.0
14,20235735,아쿠아맨과 로스트 킹덤,N,F,49368.0,4.0
1,20235596,트롤: 밴드 투게더,N,F,28988.0,5.0
7,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,11586.0,6.0
20,20235290,도티와 영원의 탑,N,K,10704.0,7.0
15,20234114,괴물,Y,F,8710.0,8.0
0,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",Y,F,6471.0,9.0
23,20235980,말하고 싶은 비밀,N,F,3765.0,10.0


In [20]:
df=pd.read_parquet("/home/jacob/data/movies/merge/dailyboxoffice/dt=20240104")
df

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
18,20235834,위시,N,F,68674,1.0
20,20212866,서울의 봄,N,K,45359,2.0
23,20203702,노량: 죽음의 바다,N,K,44860,3.0
12,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,12113,4.0
1,20228555,더 퍼스트 슬램덩크,N,F,8114,5.0
7,20235735,아쿠아맨과 로스트 킹덤,N,F,7170,6.0
21,20235583,인투 더 월드,N,F,5898,7.0
13,20234114,괴물,Y,F,3494,8.0
16,20235596,트롤: 밴드 투게더,N,F,3119,9.0
4,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,3051,10.0


In [21]:
df2=pd.read_parquet("/home/jacob/data/movies/merge/dailyboxoffice/dt=20240101/merged.parquet")
df2

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
24,20203702,노량: 죽음의 바다,N,K,290717,1.0
17,20212866,서울의 봄,N,K,262422,2.0
4,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,58239,3.0
21,20235735,아쿠아맨과 로스트 킹덤,N,F,49368,4.0
16,20235596,트롤: 밴드 투게더,N,F,28988,5.0
1,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,11586,6.0
3,20235290,도티와 영원의 탑,N,K,10704,7.0
7,20234114,괴물,Y,F,8710,8.0
10,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",Y,F,6471,9.0
11,20235980,말하고 싶은 비밀,N,F,3765,10.0


In [23]:
df3=pd.read_parquet("/home/jacob/data/movies/merge/dailyboxoffice/dt=20240106")
df3

,movieCd,movieNm,multiMovieYn,repNationCd,audiCnt,rank
23,20235834,위시,N,F,189447,1.0
15,20203702,노량: 죽음의 바다,N,K,113941,2.0
11,20212866,서울의 봄,N,K,108206,3.0
1,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,N,F,29474,4.0
4,20235735,아쿠아맨과 로스트 킹덤,N,F,17366,5.0
14,20234114,괴물,Y,F,8015,6.0
5,20228555,더 퍼스트 슬램덩크,N,F,7707,7.0
13,20235596,트롤: 밴드 투게더,N,F,6114,8.0
7,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K,5168,9.0
22,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",Y,F,2766,10.0
